In [42]:
import json
from datetime import datetime, timedelta
from shapely.geometry import LineString, Point
import folium

# Ścieżka do pliku JSON
file_path = r'A:\repo_git\aircraft_signals\dane\data.json'

# Lista do przechowywania wszystkich obiektów JSON
data = []

# Wczytaj dane z pliku data.json (każda linia to osobny obiekt JSON)
with open(file_path, 'r') as file:
    for line in file:
        try:
            # Usuń niepotrzebne znaki nowej linii lub inne zbędne znaki
            cleaned_line = line.strip()

            # Spróbuj parsować linię jako obiekt JSON
            obj = json.loads(cleaned_line)

            # Dodaj poprawnie sparsowany obiekt do listy danych
            data.append(obj)

        except json.JSONDecodeError as e:
            print(f"Błąd podczas parsowania JSON w linii: {cleaned_line[:50]}...")
            print(f"Komunikat błędu: {str(e)}")
            continue

# Lista do przechowywania identyfikatorów samolotów z potencjalną kolizją oraz czasu i położenia kolizji
collisions = []

# Przetwarzanie każdej pary lotów i sprawdzanie przecięć oraz czasu PITR
for i in range(len(data)):
    ident1 = data[i]['ident']
    waypoints1 = data[i].get('waypoints', [])
    if len(waypoints1) < 2:
        continue
    path1 = LineString([(point['lon'], point['lat']) for point in waypoints1])
    pitr1 = datetime.strptime(data[i]['pitr'], '%Y-%m-%d %H:%M:%S')

    for j in range(i + 1, len(data)):
        ident2 = data[j]['ident']
        waypoints2 = data[j].get('waypoints', [])
        if len(waypoints2) < 2:
            continue
        path2 = LineString([(point['lon'], point['lat']) for point in waypoints2])
        pitr2 = datetime.strptime(data[j]['pitr'], '%Y-%m-%d %H:%M:%S')

        # Sprawdzanie przecięć trajektorii i czasu PITR
        if path1.intersects(path2):
            time_diff = abs(pitr1 - pitr2)
            if time_diff <= timedelta(minutes=10):
                intersection_point = path1.intersection(path2)
                if intersection_point.is_empty:
                    continue
                if intersection_point.geom_type == 'Point':
                    intersection_lat = intersection_point.y
                    intersection_lon = intersection_point.x
                    collisions.append((ident1, ident2, pitr1, intersection_lat, intersection_lon, pitr2))
                break
    if len(collisions) >= 10:
        break

# Wypisanie 10 pierwszych samolotów z potencjalną kolizją
print("Samoloty z potencjalną kolizją:")
for k in range(10):
    if k < len(collisions):
        ident1, ident2, pitr1, intersection_lat, intersection_lon, pitr2 = collisions[k]
        formatted_time1 = pitr1.strftime('%Y-%m-%d %H:%M:%S')
        formatted_time2 = pitr2.strftime('%Y-%m-%d %H:%M:%S')
        print(f"{ident1}  i {ident2}")

# Stwórz mapę
mymap = folium.Map(location=[0, 0], zoom_start=2)

# Zdefiniuj unikalne kolory dla każdego samolotu
plane_colors = {}

# Kolory do oznaczenia tras samolotów
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen', 'darkpurple', 'cadetblue']

# Dodaj trajektorie samolotów z potencjalną kolizją do mapy
for idx, flight in enumerate(data):
    ident = flight['ident']
    if (ident in [collision[0] for collision in collisions[:10]] or
        ident in [collision[1] for collision in collisions[:10]]):

        waypoints = flight.get('waypoints', [])

        # Ekstrakcja współrzędnych do listy tupli (lat, lon)
        coordinates = [(point['lat'], point['lon']) for point in waypoints]

        # Wygeneruj lub pobierz kolor dla danego samolotu
        if ident not in plane_colors:
            plane_colors[ident] = colors[len(plane_colors) % len(colors)]

        # Dodaj trajektorię do mapy
        folium.PolyLine(coordinates, color=plane_colors[ident], weight=2.5, opacity=1).add_to(mymap)

        # Dodaj marker dla początkowego punktu trasy (pierwszy punkt)
        if coordinates:
            folium.Marker(location=coordinates[0], icon=folium.Icon(color=plane_colors[ident]), popup=f"Początkowy punkt: {ident}").add_to(mymap)

        # Dodaj marker dla końcowego punktu trasy (ostatni punkt)
        if coordinates:
            folium.Marker(location=coordinates[-1], icon=folium.Icon(color=plane_colors[ident]), popup=f"Końcowy punkt: {ident}").add_to(mymap)

        # Dodaj marker dla punktu przecięcia (kolizji)
        folium.Marker(location=[intersection_lat, intersection_lon], icon=folium.Icon(color='black'), popup=f"Potencjalna kolizja: {ident1} i {ident2}\nCzas: {formatted_time1}\nPołożenie: {intersection_lat}, {intersection_lon}").add_to(mymap)

# Zapisz mapę do pliku HTML
mymap.save('collision_flight_map_with_intersections.html')

print("Mapa została zapisana jako collision_flight_map_with_intersections.html")


Samoloty z potencjalną kolizją:
THY1  i AAL2254
QFA79  i KAL124
KAL124  i HAL459
AAL1203  i DAL1155
NKS957  i QXE2738
AAY751  i SWA4293
EJA782  i WJA1697
UAL1748  i SWA4155
DAL446  i KLM606
SWA4155  i AAL1632
Mapa została zapisana jako collision_flight_map_with_intersections.html
